# ETOPO1 on the sphere

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import quagmire
from petsc4py import PETSc
from scipy.spatial import cKDTree

In [ ]:
ETOPO1_file = "data/ETOPO1_Bed_g_int.xyz_100k.npz"

with np.load(ETOPO1_file, 'r') as f:
    etopo1 = f['arr_0']
    npoints = etopo1.shape[0]
    
# convert to radians
lons = etopo1[:,0]*np.pi/180 #+ np.random.random(npoints)*1e-5
lats = etopo1[:,1]*np.pi/180 #+ np.random.random(npoints)*1e-5

height = etopo1[:,2]

# below sea level
bmask = height > 0.0

In [ ]:
# We add a little noise to avoid colinearity problem
DM = quagmire.tools.create_spherical_DMPlex_from_points(lons+ np.random.random(npoints)*1e-8,
                                                        lats+ np.random.random(npoints)*1e-8,
                                                        bmask)

In [ ]:
mesh = quagmire.SurfaceProcessMesh(DM)

In [ ]:
# query KDTree to get ordering
tree = cKDTree(np.column_stack([lons, lats]))
d, idx = tree.query(np.column_stack([mesh.tri.lons, mesh.tri.lats]))

topo = height[idx]

In [ ]:
H5_name = 'spherical_mesh.h5'

mesh.save_mesh_to_hdf5(H5_name)
mesh.save_field_to_hdf5(H5_name, height=topo)
quagmire.tools.generate_xdmf(H5_name)

## Topography

In [ ]:
mesh.update_height(topo)

In [ ]:
upstream_area = mesh.cumulative_flow(np.ones(mesh.npoints))

rainfall = height**2
# rainfall[height<=0.0] = 0.0
cumulative_rain = mesh.cumulative_flow(rainfall)
cumulative_rain[height<=0] = 0.0

In [ ]:
mesh.save_field_to_hdf5(H5_name, upstream_area=upstream_area)
mesh.save_field_to_hdf5(H5_name, slope=mesh.slope)
mesh.save_field_to_hdf5(H5_name, rain=cumulative_rain)
quagmire.tools.generate_xdmf(H5_name)